In [3]:
%env WORKDIR=/tmp/vault
%env VAULT_KMIP_PORT=5696
%env MONGODB_PORT=27017

env: WORKDIR=/tmp/vault
env: VAULT_KMIP_PORT=5696
env: MONGODB_PORT=27017


In [4]:
%%bash
cat $WORKDIR/VAULT_TOKEN.TXT
cat $WORKDIR/VAULT_ADDR.TXT

hvs.QbtInWOOYqrZJnMrDmcaArKs
https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200


In [5]:
%env VAULT_ADDR=https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200
%env VAULT_TOKEN=hvs.QbtInWOOYqrZJnMrDmcaArKs

env: VAULT_ADDR=https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200
env: VAULT_TOKEN=hvs.QbtInWOOYqrZJnMrDmcaArKs


# MongoDB Enterprise without Encryption at Rest

Setup a demo MongoDB server.  Show that data stored is unencrypted by default.

In [98]:
%%bash
# Check that the MongoDB are running
# kubectl get pods -n mongodb -o yaml
export POD=$(kubectl get pods -n mongodb -l app=mongodb -o jsonpath='{.items[0].metadata.name}')

In [99]:
%%bash 
export POD=$(kubectl get pods -n mongodb -l app=mongodb -o jsonpath='{.items[0].metadata.name}')
# Show the contents of MongoDB database data folder
kubectl exec -i -t -n mongodb $POD -- ls /data/db

Unable to use a TTY - input is not a terminal or the right kind of file


WiredTiger
WiredTiger.lock
WiredTiger.turtle
WiredTiger.wt
WiredTigerHS.wt
_mdb_catalog.wt
collection-0-422369378822914931.wt
collection-2-422369378822914931.wt
collection-4-422369378822914931.wt
collection-7-422369378822914931.wt
diagnostic.data
index-1-422369378822914931.wt
index-3-422369378822914931.wt
index-5-422369378822914931.wt
index-6-422369378822914931.wt
index-8-422369378822914931.wt
index-9-422369378822914931.wt
journal
mongod.lock
sizeStorer.wt
storage.bson


In [101]:
%env MONGODB_PORT=27017

env: MONGODB_PORT=27017


In [109]:
%%bash
# Create database “record”, insert record with random user details (using function makeid), and display table records
# Note: You can run this multiple times to insert more records as required
export MONGODB_ADDR=$(kubectl get svc -n mongodb mongodb -o json | jq -r '.status.loadBalancer.ingress[0].ip')
export USERNAME="mdbadmin"
export PASSWORD="Passw0rd123!"

mongosh -u "$USERNAME" -p "$PASSWORD" "mongodb://${MONGODB_ADDR}/admin" \
--eval "use record" \
--eval "function makeid(length) {
    var result = '';
    var characters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789';
    for (var i = 0; i < length; i++) {
        result += characters.charAt(Math.floor(Math.random() * characters.length));
    }
    return result;
}" \
--eval "db.users.insertOne({username: 'user-' + makeid(4), password: makeid(20)})" \
--eval "db.users.find()"


Current Mongosh Log ID:	673c8cff1b4eb3f675968783
Connecting to:		mongodb://<credentials>@34.175.76.53/admin?directConnection=true&appName=mongosh+2.1.5


(node:99679) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


Using MongoDB:		7.0.15
Using Mongosh:		2.1.5

For mongosh info see: https://docs.mongodb.com/mongodb-shell/

------
   The server generated these startup warnings when booting
   2024-11-19T11:31:48.062+00:00: Using the XFS filesystem is strongly recommended with the WiredTiger storage engine. See http://dochub.mongodb.org/core/prodnotes-filesystem
   2024-11-19T11:31:49.135+00:00: vm.max_map_count is too low
------

[
  {
    _id: ObjectId('673c8cff1b4eb3f675968784'),
    username: 'user-4saH',
    password: '5NTcE1pGzqilJC0t3Cpz'
  }
]


In [110]:
%%bash
export MONGODB_ADDR=$(kubectl get svc -n mongodb mongodb -o json | jq -r '.status.loadBalancer.ingress[0].ip')
export USERNAME="mdbadmin"
export PASSWORD="Passw0rd123!"

# Flush all data to the storage layer
export MONGODB_ADDR=$(kubectl get svc -n mongodb mongodb -o json | jq -r '.status.loadBalancer.ingress[0].ip')
# Take note of the username and password values being inserted into the table
mongosh -u "$USERNAME" -p "$PASSWORD" "mongodb://${MONGODB_ADDR}/admin" --eval "db.adminCommand( { fsync: 1 } )"

Current Mongosh Log ID:	673c8d2dc76d359d37ec0ba4
Connecting to:		mongodb://<credentials>@34.175.76.53/admin?directConnection=true&appName=mongosh+2.1.5


(node:302) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


Using MongoDB:		7.0.15
Using Mongosh:		2.1.5

For mongosh info see: https://docs.mongodb.com/mongodb-shell/

------
   The server generated these startup warnings when booting
   2024-11-19T11:31:48.062+00:00: Using the XFS filesystem is strongly recommended with the WiredTiger storage engine. See http://dochub.mongodb.org/core/prodnotes-filesystem
   2024-11-19T11:31:49.135+00:00: vm.max_map_count is too low
------

{ numFiles: 1, ok: 1 }


In [111]:
%%bash
# Now view the contents of MongoDB database data folder again.  You should notice a new collection file for the data we inserted. 
# It should be named like "collection-7--XXXXXXXXXXXXXXXXXX.wt" collection-7--4848515339651436823.wt 
export POD=$(kubectl get pods -n mongodb -l app=mongodb -o jsonpath='{.items[0].metadata.name}')
# Show the contents of MongoDB database data folder
echo "All Files"
kubectl exec -i -t -n mongodb $POD -- ls /data/db 

All Files


Unable to use a TTY - input is not a terminal or the right kind of file


WiredTiger
WiredTiger.lock
WiredTiger.turtle
WiredTiger.wt
WiredTigerHS.wt
_mdb_catalog.wt
collection-0-2562801038321525715.wt
collection-0-422369378822914931.wt
collection-2-422369378822914931.wt
collection-4-422369378822914931.wt
collection-7-422369378822914931.wt
diagnostic.data
index-1-2562801038321525715.wt
index-1-422369378822914931.wt
index-3-422369378822914931.wt
index-5-422369378822914931.wt
index-6-422369378822914931.wt
index-8-422369378822914931.wt
index-9-422369378822914931.wt
journal
mongod.lock
sizeStorer.wt
storage.bson


In [112]:
%%bash
# Now view the contents of MongoDB database data folder again.  You should notice a new collection file for the data we inserted. 
# It should be named like "collection-7--XXXXXXXXXXXXXXXXXX.wt" collection-7--4848515339651436823.wt 
export POD=$(kubectl get pods -n mongodb -l app=mongodb -o jsonpath='{.items[0].metadata.name}')
# Show the contents of MongoDB database data folder
echo "Filtering for collection-7-XXXXXXXXXXXXXXXXXX.wt"
kubectl exec -i -t -n mongodb $POD  -- ls /data/db | grep "collection-7"

Filtering for collection-7-XXXXXXXXXXXXXXXXXX.wt


Unable to use a TTY - input is not a terminal or the right kind of file


collection-7-422369378822914931.wt


# MongoDB Enterprise with Encryption at Rest with Vault KMIP Engine

This section demonstrates the configuration of MongoDB Enterprise with Encryption at Rest with HashiCorp's Vault KMIP secrets engine.

Ref:
- https://developer.hashicorp.com/vault/docs/secrets/kmip
- https://developer.hashicorp.com/vault/tutorials/adp/kmip-engine

## Step 1 - Enable Vault KMIP Engine

In [50]:
%%bash
# Enable KMIP secrets engine at the default mount path
#vault secrets disable kmip
vault secrets enable kmip

Success! Enabled the kmip secrets engine at: kmip/


In [6]:
%%bash
# Start KMIP Server and set client TLS certificate TTL to 365 days

# Get the docker IP address of the Vault server to configure the certificate SANs.  Otherwise the MongoDB KMIP connection will fail.
export KMIP_ADDR=$(terraform  output -state=../1_Platform_Deployment/terraform.tfstate -raw cluster_primary_fqdn_kmip)
export FQDN=$(echo "$KMIP_ADDR" | sed -E 's~https://([^:/]+).*~\1~')
export IP=$(dig $FQDN +short)


# Option 1 - Elliptic Curve (EC) is the default
vault write kmip/config listen_addrs=0.0.0.0 default_tls_client_ttl=365d server_hostnames=$FQDN,localhost server_ips=$IP,127.0.0.1
# Option 2 - RSA
#vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT tls_ca_key_type="rsa" tls_ca_key_bits=2048 default_tls_client_ttl=365d server_hostnames=$VAULT_IP,localhost

# Check the KMIP configuration
echo
vault read kmip/config

Success! Data written to: kmip/config

Key                            Value
---                            -----
default_tls_client_key_bits    256
default_tls_client_key_type    ec
default_tls_client_ttl         8760h
listen_addrs                   [0.0.0.0]
server_hostnames               [vault-primary-europe-southwest1-kmip-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com localhost]
server_ips                     [34.175.74.183 127.0.0.1]
tls_ca_key_bits                256
tls_ca_key_type                ec
tls_min_version                tls12


## Step 2 - Create KMIP Scope and Role in Vault

The KMIP secrets engine uses the concept of scopes to partition KMIP managed object storage into multiple named buckets. Within a scope, roles can be created which dictate the set of allowed operations that the particular role can perform. TLS client certificates can be generated for a role, which services and applications can then use when sending KMIP requests against Vault's KMIP secret engine.

In order to generate client certificates for KMIP clients to interact with Vault's KMIP server, we must first create a scope and role and specify the desired set of allowed operations for it.

In [66]:
%env KMIP_SVC_NAME=mongodb-svc
%env KMIP_ROLE_NAME=tde
%env CERT_PATH=certs

env: KMIP_SVC_NAME=mongodb-svc
env: KMIP_ROLE_NAME=tde
env: CERT_PATH=certs


In [63]:
%%bash
# Settings for the KMIP scope

# Create a scope
vault write -f kmip/scope/$KMIP_SVC_NAME

# List scopes and verify the KMIP scope has been created
echo
vault list kmip/scope

Success! Data written to: kmip/scope/mongodb-svc

Keys
----
mongodb-svc


In [64]:
%%bash
# Settings for the KMIP role to be created

# Create the KMIP Role (note the TTL if not set, it uses the default Vault token/lease TTL of 768hrs)
vault write kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME tls_client_key_bits=2048 tls_client_key_type=rsa operation_all=true tls_client_ttl=365d

# List roles and verify the KMIP role has been created
echo
vault list kmip/scope/$KMIP_SVC_NAME/role

Success! Data written to: kmip/scope/mongodb-svc/role/tde

Keys
----
tde


In [65]:
%%bash
# Optional - View the role details
vault read kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME

Key                    Value
---                    -----
operation_all          true
tls_client_key_bits    2048
tls_client_key_type    rsa
tls_client_ttl         8760h


## Step 3 - Create the KMIP certificates for MongoDB Encryption at Rest

In [82]:
%%bash
# Specify certificate folder path


# Create a folder for the KMIP certificates
mkdir $CERT_PATH

# Save KMIP CA certificate
vault read -format=json kmip/ca | jq -r .data.ca_pem > $CERT_PATH/ca.pem    

# Generate the required certificates
vault write -format=json \
  kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME/credential/generate \
  format=pem > credential.json

# Save the certificate and private key
jq -r .data.certificate < credential.json > $CERT_PATH/cert.pem
jq -r .data.private_key < credential.json > $CERT_PATH/key.pem
cat $CERT_PATH/cert.pem $CERT_PATH/key.pem > $CERT_PATH/client.pem

# Remove the temp file
rm credential.json

mkdir: certs: File exists


## Step 4 - Configuring MongoDB Enterprise Server with Encryption at Rest

In [83]:
%%bash
export KMIP_ADDR=$(terraform  output -state=../1_Platform_Deployment/terraform.tfstate -raw cluster_primary_fqdn_kmip)
export FQDN=$(echo "$KMIP_ADDR" | sed -E 's~https://([^:/]+).*~\1~')
export IP=$(dig $FQDN +short)

# Start a MongoDB node 
docker run -d --rm --volume=./certs:/certs -p 27017:27017 --name mongo-enterprise \
mongodb/mongodb-enterprise-server:7.0-ubuntu2204 --port 27017 --bind_ip "0.0.0.0" \
--enableEncryption --kmipServerName $IP --kmipPort $VAULT_KMIP_PORT \
--kmipServerCAFile /certs/ca.pem --kmipClientCertificateFile /certs/client.pem

ad1c9534366b55cf01fed828765fabb0c870c8b5634af75861b2266ae624eb22


In [85]:
# Check that the MongoDB are running
! docker ps

CONTAINER ID   IMAGE                                              COMMAND                  CREATED          STATUS          PORTS                      NAMES
ad1c9534366b   mongodb/mongodb-enterprise-server:7.0-ubuntu2204   "python3 /usr/local/…"   42 seconds ago   Up 42 seconds   0.0.0.0:27017->27017/tcp   mongo-enterprise
a2a79f0c4a47   hashicorp/vault-enterprise:latest                  "docker-entrypoint.s…"   18 hours ago     Up 18 hours     0.0.0.0:8200->8200/tcp     vault-enterprise


## Step 5 - Testing MongoDB Enterprise Encryption at Rest

In [89]:
%%bash
# Create database “record”, insert record with random user details (using function makeid), and display table records
# Note: You can run this multiple times to insert more records as required

# Take note of the username and password values being inserted into the table
mongosh mongodb://127.0.0.1:$MONGODB_PORT/admin --eval "use record" \
--eval "function makeid(length) {
    var result           = '';
    var characters       = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789';
    var charactersLength = characters.length;
    for ( var i = 0; i < length; i++ ) {
      result += characters.charAt(Math.floor(Math.random() * 
 charactersLength));
   }
   return result;
}" \
--eval "db.users.insertOne({username: \"user-\" + makeid(4), password: makeid(20)})" \
--eval "db.users.find()"

Current Mongosh Log ID:	673c5d35e451c8093b0433d4
Connecting to:		mongodb://127.0.0.1:27017/admin?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.1.5


(node:4456) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


Using MongoDB:		7.0.15
Using Mongosh:		2.1.5

For mongosh info see: https://docs.mongodb.com/mongodb-shell/

------
   The server generated these startup warnings when booting
   2024-11-19T09:37:21.801+00:00: Using the XFS filesystem is strongly recommended with the WiredTiger storage engine. See http://dochub.mongodb.org/core/prodnotes-filesystem
   2024-11-19T09:37:22.698+00:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted
   2024-11-19T09:37:22.698+00:00: /sys/kernel/mm/transparent_hugepage/enabled is 'always'. We suggest setting it to 'never' in this binary version
   2024-11-19T09:37:22.699+00:00: vm.max_map_count is too low
------

[
  {
    _id: ObjectId('673c5d36e451c8093b0433d5'),
    username: 'user-BxCT',
    password: '3fBQXJ7dlcGsGOiDRzbf'
  }
]


In [90]:
%%bash
# Simulate that the KMIP server is down by changing the listening port to 5697
vault write kmip/config listen_addrs=0.0.0.0:5697

Success! Data written to: kmip/config


In [91]:
%%bash
# Reinsert another record and query to show that CRUD operations are still working
mongosh mongodb://127.0.0.1:$MONGODB_PORT/admin --eval "use record" \
--eval "function makeid(length) {
    var result           = '';
    var characters       = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789';
    var charactersLength = characters.length;
    for ( var i = 0; i < length; i++ ) {
      result += characters.charAt(Math.floor(Math.random() * 
 charactersLength));
   }
   return result;
}" \
--eval "db.users.insertOne({username: \"user-\" + makeid(4), password: makeid(20)})" \
--eval "db.users.find()"

Current Mongosh Log ID:	673c5d744a8d8215629fef37
Connecting to:		mongodb://127.0.0.1:27017/admin?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.1.5


(node:5204) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


Using MongoDB:		7.0.15
Using Mongosh:		2.1.5

For mongosh info see: https://docs.mongodb.com/mongodb-shell/

------
   The server generated these startup warnings when booting
   2024-11-19T09:37:21.801+00:00: Using the XFS filesystem is strongly recommended with the WiredTiger storage engine. See http://dochub.mongodb.org/core/prodnotes-filesystem
   2024-11-19T09:37:22.698+00:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted
   2024-11-19T09:37:22.698+00:00: /sys/kernel/mm/transparent_hugepage/enabled is 'always'. We suggest setting it to 'never' in this binary version
   2024-11-19T09:37:22.699+00:00: vm.max_map_count is too low
------

[
  {
    _id: ObjectId('673c5d36e451c8093b0433d5'),
    username: 'user-BxCT',
    password: '3fBQXJ7dlcGsGOiDRzbf'
  },
  {
    _id: ObjectId('673c5d754a8d8215629fef38'),
    username: 'user-B51q',
    password: 'andNrCvNDgcSTCS5BApK'
  }
]


In [92]:
# Flush all data to the storage layer
! mongosh mongodb://127.0.0.1:$MONGODB_PORT/admin --eval "db.adminCommand( { fsync: 1 } )"

]0;mongosh mongodb://127.0.0.1:27017/admin?directConnection=true&serverSelectionTimeoutMS=2000Current Mongosh Log ID:	673c5d8edf25ebac848add67
Connecting to:		mongodb://127.0.0.1:27017/admin?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.1.5
(node:5479) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)
Using MongoDB:		7.0.15
Using Mongosh:		2.1.5

For mongosh info see: https://docs.mongodb.com/mongodb-shell/

 ------
   The server generated these startup warnings when booting
   2024-11-19T09:37:21.801+00:00: Using the XFS filesystem is strongly recommended with the WiredTiger storage engine. See http://dochub.mongodb.org/core/prodnotes-filesystem
   2024-11-19T09:37:22.698+00:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted
   2024-11-19T09:37:22.698+00:00: /sys

# Cleanup

In [ ]:
# Cleanup

# Disable KMIP secrets engine
vault secrets disable kmip

# Stop Vault container
docker stop vault-enterprise

# Stop MongoDB container
docker stop mongo-enterprise

# Remove KMIP certificate demo files
rm -rf $CERT_PATH